In [2]:
from transform import preprocessing

df = preprocessing(20)

Data has been written to data/preprocessed/20-round-preprocessed.csv


In [3]:
print(df['구단'].value_counts())

구단
김천      40
광주      38
대전      38
천안      35
전북      35
수원      35
포항      34
서울      34
울산      33
인천      33
전남      33
강원      33
충남아산    32
성남      32
제주      32
경남      31
수원FC    31
서울E     31
김포      31
안양      31
대구      31
부천      30
충북청주    29
안산      29
부산      28
Name: count, dtype: int64


In [5]:
print(len(df.columns))

134


In [38]:
import numpy as np
from scipy.stats import rankdata
import pandas as pd
import plotly.graph_objects as go

class RadarChart(object):
    def __init__(self, player_name, df):
        self.fig = go.Figure()
        self.df = df
        self.player_name = player_name
        self.player_position = self.df[df['선수명'] == player_name]["포지션"].values[0]
        self.string_column = ['index', '선수명', '구단', '포지션', '등번호']
        self.radar_columns = ['득점/xG', '슈팅/90', '유효 슈팅/90', '드리블 시도/90', '공격지역패스 성공%', '키패스/90', 'Defensive Action/90', 'Possession won/90', '경합 공중 성공/90']

        # 특정 선수의 데이터프레임 필터링
        self.relative_df = df[df['포지션'] == self.player_position]
        self.relative_df = self.relative_df[self.relative_df['출전시간(분)'] >= 450]
        
        # 퍼센타일 스케일링
        self.radar_scaled_df = self.relative_df.copy()
        self.radar_scaled_columns = self.relative_df[self.radar_columns]
        
        def percentile_rank(x):
            # 모든 값이 동일한 경우를 처리
            if len(np.unique(x)) == 1:
                return np.zeros(len(x))
            ranks = rankdata(x)
            scaled = 100 * (ranks - 1) / (len(ranks) - 1)
            return scaled
        
        self.radar_scaled_df[self.radar_columns] = self.radar_scaled_columns.apply(percentile_rank)
        
        # 특정 선수의 데이터 추출
        self.target_df = self.radar_scaled_df[self.radar_scaled_df["선수명"] == player_name][self.radar_columns]
        self.key_value = '키패스/90'
        print(self.radar_scaled_df.sort_values(by=self.key_value, ascending=False).head(10))

    def plot(self):
        if self.target_df.empty:
            print(f"No data available for player {self.player_name}")
            return

        categories = self.radar_columns
        values = self.target_df.values.flatten().tolist()

        # Close the loop
        values += values[:1]
        categories += categories[:1]

        self.fig.add_trace(go.Scatterpolar(
            r=values,
            theta=categories,
            fill='toself',
            name=self.player_name
        ))

        self.fig.update_layout(
            polar=dict(
                radialaxis=dict(
                    visible=True,
                    range=[0, 100]
                )),
            showlegend=True,
            title=f"Radar Chart for {self.player_name}"
        )

        self.fig.show()



In [41]:

mh_radar = RadarChart('이승우', df)
mh_radar.plot()


     index   선수명    구단 포지션   등번호  출전시간(분)   득점   도움    슈팅  유효 슈팅  ...  \
495  491.0   이동경    울산  MF  14.0    611.0  7.0  5.0  24.0   14.0  ...   
733  727.0   최우진    인천  MF  14.0   1028.0  0.0  3.0   7.0    2.0  ...   
211  209.0   라마스    부산  MF  10.0   1765.0  4.0  4.0  68.0   22.0  ...   
750  744.0   카즈키    수원  MF  14.0    805.0  0.0  2.0  11.0    4.0  ...   
123  122.0   김영욱   서울E  MF  14.0    497.0  0.0  0.0   2.0    0.0  ...   
356  352.0   송민규    전북  MF  10.0   1345.0  3.0  6.0  18.0    6.0  ...   
761  755.0    페신    부산  MF   7.0   1495.0  4.0  2.0  31.0   13.0  ...   
387  383.0  안데르손  수원FC  MF  70.0   1782.0  1.0  8.0  49.0   14.0  ...   
213  211.0   로페즈    부산  MF  11.0   1138.0  0.0  2.0  34.0   15.0  ...   
224  222.0  마테우스    안양  MF   7.0   1642.0  4.0  4.0  50.0   24.0  ...   

        획득/90     블락/90    볼미스/90     파울/90    피파울/90     경고/90     퇴장/90  \
495  5.302782  1.325696  0.883797  1.031097  0.589198  0.000000  0.000000   
733  6.828794  1.488327  0.437743  0.26264